<a href="https://colab.research.google.com/github/aneeq-shaffy/DL-labsheets/blob/main/Pest_solution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import kagglehub

# Download and cache the dataset
path = kagglehub.dataset_download("imbikramsaha/paddy-doctor")

# Print dataset path
print("Path to dataset files:", path)


Using Colab cache for faster access to the 'paddy-doctor' dataset.
Path to dataset files: /kaggle/input/paddy-doctor


In [9]:
import os

# Define the root path of the downloaded dataset
dataset_root_path = '/kaggle/input/paddy-doctor'
print(f"Listing contents of: {dataset_root_path}")

# Walk through the directory and print files and subdirectories
for dirpath, dirnames, filenames in os.walk(dataset_root_path):
    for d in dirnames:
        print(os.path.join(dirpath, d) + " (directory)")
    for f in filenames:
        print(os.path.join(dirpath, f))


Streaming output truncated to the last 5000 lines.
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/102529.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/109282.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/101803.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/104399.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/101546.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/105086.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/109543.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/106919.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/110268.jpg
/kaggle/input/paddy-doctor/paddy-disease-classification/train_images/dead_heart/103727.jpg
/kaggle/input/paddy-doctor/paddy-diseas

In [10]:
import pandas as pd
import os

# Paths
dataset_root_path = os.path.join(path, 'paddy-disease-classification')
train_csv_path = os.path.join(dataset_root_path, 'train.csv')

# Load train.csv if it exists
if os.path.exists(train_csv_path):
    train_df = pd.read_csv(train_csv_path)
    print("First 5 rows of train.csv:")
    display(train_df.head())
else:
    print(f"Error: train.csv not found at {train_csv_path}")


First 5 rows of train.csv:


,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


In [11]:
# Extract unique label classes
unique_labels = train_df['label'].unique()
print("Unique label classes:")
for label in unique_labels:
    print(label)


Unique label classes:
bacterial_leaf_blight
bacterial_leaf_streak
bacterial_panicle_blight
blast
brown_spot
dead_heart
downy_mildew
hispa
normal
tungro


In [12]:
# Display the number of images per class
print("Image counts per class:")
label_counts = train_df['label'].value_counts()
display(label_counts)


Image counts per class:


,count
label,
normal,1764
blast,1738
hispa,1594
dead_heart,1442
tungro,1088
brown_spot,965
downy_mildew,620
bacterial_leaf_blight,479
bacterial_leaf_streak,380


In [15]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

counts = torch.tensor(label_counts.sort_index().values, dtype=torch.float)
class_weights = 1.0 / counts
class_weights = class_weights / class_weights.sum()
class_weights = class_weights.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)


Using device: cpu


In [16]:
import os
from PIL import Image

dataset_root_path = os.path.join(path, 'paddy-disease-classification')
train_images_path = os.path.join(dataset_root_path, 'train_images')

class_dirs = [d for d in os.listdir(train_images_path) if os.path.isdir(os.path.join(train_images_path, d))]

if not class_dirs:
    print(f"No class directories found in {train_images_path}")
else:
    sample_class = class_dirs[0]
    sample_class_path = os.path.join(train_images_path, sample_class)

    sample_images = [f for f in os.listdir(sample_class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if not sample_images:
        print(f"No images found in class '{sample_class}'")
    else:
        first_image_path = os.path.join(sample_class_path, sample_images[0])
        try:
            with Image.open(first_image_path) as img:
                print(f"Sample Image: {sample_images[0]}")
                print(f"Path: {first_image_path}")
                print(f"Size: {img.size}, Format: {img.format}, Mode: {img.mode}")
        except Exception as e:
            print(f"Error opening image {first_image_path}: {e}")


Sample Image: 109629.jpg
Path: /kaggle/input/paddy-doctor/paddy-disease-classification/train_images/tungro/109629.jpg
Size: (480, 640), Format: JPEG, Mode: RGB


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

path = '/root/.cache/kagglehub/datasets/imbikramsaha/paddy-doctor/versions/1'
dataset_root_path = os.path.join(path, 'paddy-disease-classification')
train_csv_path = os.path.join(dataset_root_path, 'train.csv')

full_df = pd.read_csv(train_csv_path)

train_df, val_df = train_test_split(
    full_df,
    test_size=0.1,
    stratify=full_df['label'],
    random_state=42
)

print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")
val_df.head()


Training samples: 9366, Validation samples: 1041


,image_id,label,variety,age
8509,109151.jpg,normal,ADT45,65
2009,102055.jpg,blast,AndraPonni,45
5530,100170.jpg,downy_mildew,ADT45,70
4787,108524.jpg,dead_heart,ADT45,75
7864,101175.jpg,normal,ADT45,60


In [18]:
import torchvision.transforms as transforms

train_transform = transforms.Compose([
    # 1. Resize images to 224x224 (EfficientNet input size)
    transforms.Resize((224, 224)),

    # 2. Random horizontal and vertical flips
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),

    # 3. Random rotation for robustness (±20°)
    transforms.RandomRotation(20),

    # 4. Color jitter to handle lighting variations
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),

    # 5. Convert PIL image to PyTorch tensor (scales 0-255 → 0-1)
    transforms.ToTensor(),

    # 6. Normalize using ImageNet stats (needed for pretrained EfficientNet)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("Training and validation transforms defined.")


Training and validation transforms defined.


In [19]:
from torch.utils.data import DataLoader
import os

train_images_path = os.path.join(dataset_root_path, 'train_images')

# Training dataset with optional rare-class augmentation
train_dataset = PaddyDiseaseDataset(
    dataframe=train_df,
    img_dir=train_images_path,
    transform=train_transform,
    rare_class_transform=rare_class_transform
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# Validation dataset (no heavy augmentation)
val_dataset = PaddyDiseaseDataset(
    dataframe=val_df,
    img_dir=train_images_path,
    transform=val_transform
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")
print(f"Training batches: {len(train_dataloader)}, Validation batches: {len(val_dataloader)}")


Training samples: 9366, Validation samples: 1041
Training batches: 293, Validation batches: 33


In [20]:
import torchvision.transforms as transforms

resize_size = (224, 224)

train_transform = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

rare_class_transform = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.RandomHorizontalFlip(0.7),
    transforms.RandomVerticalFlip(0.7),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(resize_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [21]:
from torch.utils.data import DataLoader
import os

train_images_path = os.path.join(dataset_root_path, 'train_images')

# Training dataset + loader
train_dataset = PaddyDiseaseDataset(
    dataframe=train_df,
    img_dir=train_images_path,
    transform=train_transform,
    rare_class_transform=rare_class_transform
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# Validation dataset + loader
val_dataset = PaddyDiseaseDataset(
    dataframe=val_df,
    img_dir=train_images_path,  # same folder, split from train_df
    transform=val_transform
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Training samples: {len(train_dataset)}, Batches: {len(train_dataloader)}")
print(f"Validation samples: {len(val_dataset)}, Batches: {len(val_dataloader)}")


Training samples: 9366, Batches: 293
Validation samples: 1041, Batches: 33


In [22]:
import torch.nn as nn
import torchvision.models as models

# 1. Load the pre-trained EfficientNetB1 model
#    Use ImageNet weights for transfer learning
model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)

# 2. Freeze all parameters in the feature extractor
#    Only the new classifier will train initially
for param in model.parameters():
    param.requires_grad = False

# 3. Replace the original classifier (top layer) with a new one for your dataset
#    Get the number of input features to the classifier
in_features = model.classifier[1].in_features

#    Get number of unique classes from your dataset
num_classes = len(unique_labels)  # assuming unique_labels = train_df['label'].unique()

#    Define a new classifier
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),   # regularization
    nn.Linear(in_features, num_classes)  # output layer for your classes
)

# 4. Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 5. Print model summary
print(model)
print(f"\nEfficientNetB1 model loaded with pretrained ImageNet weights. Classifier replaced for {num_classes} classes.")


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1️⃣ Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2️⃣ Move model to device
model = model.to(device)

# 3️⃣ Loss function with class weights (for imbalanced dataset)
#    Assume 'class_weights' tensor has been computed and moved to device
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 4️⃣ Optimizer: AdamW is often better for fine-tuning pretrained models
#    Only updates trainable parameters (classifier head, rest are frozen)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# 5️⃣ Learning rate scheduler: Cosine annealing improves convergence
#    Reduces LR from initial to near-zero over num_epochs
from torch.optim.lr_scheduler import CosineAnnealingLR
num_epochs = 10  # adjust based on your training plan
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

print("Model, weighted Loss Function, AdamW Optimizer, and LR Scheduler configured.")


Using device: cpu
Model, weighted Loss Function, AdamW Optimizer, and LR Scheduler configured.


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

# ---------- Configuration ----------
num_epochs = 15
batch_size = 32
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)

# ---------- Device ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

# ---------- Optimizer & Scheduler ----------
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# ---------- Metrics ----------
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return (preds == labels).float().mean()

# ---------- Tracking ----------
train_losses, val_losses = [], []
train_accs, val_accs = [], []
best_val_loss = float('inf')

# ---------- Training Loop ----------
for epoch in range(num_epochs):
    model.train()
    running_loss, running_acc = 0.0, 0.0

    for inputs, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, labels).item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_acc / len(train_dataset)
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)

    # ---------- Validation ----------
    model.eval()
    val_running_loss, val_running_acc = 0.0, 0.0
    with torch.no_grad():
        for inputs, labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)
            val_running_acc += accuracy(outputs, labels).item() * inputs.size(0)

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = val_running_acc / len(val_dataset)
    val_losses.append(val_epoch_loss)
    val_accs.append(val_epoch_acc)

    # Step scheduler
    scheduler.step()

    print(f"\nEpoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

    # Save best model
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pth"))
        print(f"Saved best model at epoch {epoch+1}")

# ---------- Optional: Plot training curves ----------
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.title("Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_accs, label="Train Acc")
plt.plot(val_accs, label="Val Acc")
plt.title("Accuracy Curve")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

print("Training complete!")


Using device: cpu


Epoch 1/15 [Train]:   0%|          | 0/293 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 1/15 [Train]:  56%|█████▌    | 163/293 [14:33<11:36,  5.36s/it]


KeyboardInterrupt: 

In [37]:
from torch.utils.data import DataLoader

# Validation dataset using the same PaddyDiseaseDataset class
val_dataset = PaddyDiseaseDataset(
    dataframe=val_df,         # your validation DataFrame
    img_dir=train_images_path, # same root as training, since val_df points to training images
    transform=val_transform    # simple resize + normalize
)

# Validation DataLoader
val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,   # can be same as training or larger if memory allows
    shuffle=False,   # no shuffle for validation
    num_workers=2,
    pin_memory=True
)

print(f"Number of samples in validation dataset: {len(val_dataset)}")
print(f"Number of batches in validation DataLoader: {len(val_dataloader)}")


NameError: name 'val_dataloader' is not defined

In [ ]:
import torch

def validate(model, val_loader, criterion, device):
    model.eval()  # set model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # disable gradient computation
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Accumulate loss
            running_loss += loss.item() * inputs.size(0)

            # Compute accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    return epoch_loss, epoch_acc

# Example usage inside your training loop
for epoch in range(num_epochs):
    # ----- Training loop (your existing code) -----
    model.train()
    running_train_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * inputs.size(0)
    train_loss = running_train_loss / len(train_dataset)

    # ----- Validation -----
    val_loss, val_acc = validate(model, val_dataloader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc*100:.2f}%")
